# 🎬 UVG MAX
---

In [ ]:
# Step 1: Clone & Setup
import os, sys

# Clear proxy settings that may interfere with Gemini
for var in ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY']:
    os.environ.pop(var, None)

os.chdir('/content')
!rm -rf uvg-max-engine
!git clone https://github.com/shya-bubu/uvg-max-engine.git
os.chdir('uvg-max-engine')
sys.path.insert(0, '/content/uvg-max-engine')

!pip install -q python-dotenv requests tqdm torch google-generativeai==0.8.3
!pip install -q azure-cognitiveservices-speech opencv-python Pillow numpy
!pip install -q librosa pyloudnorm moviepy scikit-learn

print("✅ Ready!")

In [ ]:
# Step 2: API Keys
import os

# Clear proxy again
for var in ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY']:
    os.environ.pop(var, None)

# ============================================
# 🔐 PASTE YOUR API KEYS
# ============================================
os.environ["GEMINI_API_KEY"] = ""  # Required
os.environ["PEXELS_KEY"] = ""       # Required
os.environ["AZURE_SPEECH_KEY"] = "" # Optional
os.environ["AZURE_SPEECH_REGION"] = "eastus"
os.environ["UVG_MOCK_MODE"] = "false"
# ============================================

print("Gemini:", "✅" if os.environ["GEMINI_API_KEY"] else "❌")
print("Pexels:", "✅" if os.environ["PEXELS_KEY"] else "❌")

In [ ]:
# Step 3: Test Gemini API directly
import os
for var in ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY']:
    os.environ.pop(var, None)

import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel('gemini-2.5-flash')
response = model.generate_content("Say hello in 3 words")
print("Gemini says:", response.text)
print("✅ Gemini API works!")

In [ ]:
# Step 4: Generate Video
from uvg_core.uvg_pipeline import UVGPipeline, PipelineConfig
import os

TOPIC = "The beauty of nature"

config = PipelineConfig(
    style_pack="cinematic",
    mock_mode=os.environ.get("UVG_MOCK_MODE", "true") == "true"
)

pipeline = UVGPipeline(config)
result = pipeline.run(topic=TOPIC)

print(f"{'✅ Success' if result.success else '❌ Failed'}: {result.output_path}")
if result.errors:
    for e in result.errors:
        print(f"Error: {e}")

In [ ]:
# Step 5: Watch
from IPython.display import Video, display
import glob

videos = glob.glob("uvg_output/**/*.mp4", recursive=True)
if videos:
    display(Video(videos[0], embed=True, width=400))
else:
    print("No video found")